In [ ]:
from flask import Flask, render_template, Response, request, redirect, url_for
import cv2
import os
from ultralytics import YOLO
import math

In [ ]:
def image_detect(image_path):
    model = YOLO("runs/detect/train/weights/best.pt")
    classNames = ["bouledogue","blacktip","whitetip"]

    img = cv2.imread(image_path)
    if img is None:
        return None
    results = model(img)
    for result in results:
        boxes = result.boxes
        for box in boxes:
            conf = math.ceil((box.conf[0] * 100)) / 100
            if conf > 0.5:
                x1, y1, x2, y2 = box.xyxy[0].int().tolist()
                cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 3)
                cls = int(box.cls[0])
                class_name = classNames[cls]
                label = f'{class_name} : {conf}'
                t_size = cv2.getTextSize(label, 0, fontScale=1, thickness=2)[0]
                c2 = x1 + t_size[0], y1 - t_size[1] - 3
                cv2.rectangle(img, (x1,y1), c2, [0,255,0], -1, cv2.LINE_AA)
                cv2.putText(img, label, (x1, y1-2), 0, 1, [255,255,255], thickness=1, lineType=cv2.LINE_AA)
    result_path = image_path.replace('uploads', 'results')
    cv2.imwrite(result_path, img)
    return result_path

In [ ]:
def video_detection(path):
    """
    Detects objects in a video using YOLO model.

    Args:
        path (str): The path to the video file.

    Yields:
        numpy.ndarray: The processed frame with object detection annotations.
    """
    cap = cv2.VideoCapture(path)
    frame_width = int(cap.get(3))
    frame_height = int(cap.get(4))
    
    model = YOLO("runs/detect/train/weights/best.pt")
    classNames = ["bouledogue", "blacktip", "whitetip"]
    
    while True:
        success, img = cap.read()
        results = model(img, stream=True)
        
        for r in results:
            boxes = r.boxes
            for box in boxes:
                conf = math.ceil((box.conf[0] * 100)) / 100
                if conf > 0.5:
                    x1, y1, x2, y2 = box.xyxy[0]
                    x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
                    cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 3)
                    cls = int(box.cls[0])
                    class_name = classNames[cls]
                    label = f'{class_name} : {conf}'
                    t_size = cv2.getTextSize(label, 0, fontScale=1, thickness=2)[0]
                    c2 = x1 + t_size[0], y1 - t_size[1] - 3
                    cv2.rectangle(img, (x1, y1), c2, [0, 255, 0], -1, cv2.LINE_AA)
                    cv2.putText(img, label, (x1, y1 - 2), 0, 1, [255, 255, 255], thickness=1, lineType=cv2.LINE_AA)
        
        yield img

In [ ]:
app = Flask(__name__) 

app.config['UPLOAD_FOLDER'] = 'static/uploads/'
app.config['RESULT_FOLDER'] = 'static/results/'

def generate_frames(path = ''):
    yolo_output = video_detection(path)
    for detection in yolo_output :
        ref, buffer = cv2.imencode('.jpg', detection)
        frame = buffer.tobytes()
        yield (b'--frame\r\n' b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')

@app.route('/galerie')
def galerie():
    return render_template('galerie.html')

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/', methods=['POST'])
def upload_image():
    file = request.files['file']
    if file :
        filename = file.filename
        filepath = os.path.join(app.config['UPLOAD_FOLDER'], filename)
        file.save(filepath)
        image_detect(filepath)
        return render_template('index.html', filename=filename)
    else:
        return redirect(request.url)

@app.route('/video')
def video():
    #return Response(generate_frames(path=mavideo), mimetype='multipart/x-mixed-replace; boundary=frame')
    pass
    
@app.route('/webcam')
def webcam():
    return Response(generate_frames(path=0), mimetype='multipart/x-mixed-replace; boundary=frame')

@app.route('/display/<filename>')
def display_image(filename):
    return redirect(url_for('static', filename=f'results/{filename}', code = 301))

if __name__ == '__main__':
    app.run(debug=False)